In [1]:
import torch
import torch.nn as nn
import math

In [2]:
# Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)  # shape: (1, max_len, d_model)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)].to(x.device)
        return x

In [3]:
# Multi-Head Attention
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        assert d_model % num_heads == 0
        self.d_k = d_model // num_heads
        self.num_heads = num_heads

        self.qkv_proj = nn.Linear(d_model, d_model * 3)
        self.out_proj = nn.Linear(d_model, d_model)

    def forward(self, x):
        batch_size, seq_len, d_model = x.size()
        qkv = self.qkv_proj(x)  # shape: (batch, seq_len, 3*d_model)
        qkv = qkv.reshape(batch_size, seq_len, 3, self.num_heads, self.d_k)
        qkv = qkv.permute(2, 0, 3, 1, 4)  # (3, batch, heads, seq_len, d_k)
        Q, K, V = qkv[0], qkv[1], qkv[2]

        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        attn = torch.softmax(scores, dim=-1)
        context = torch.matmul(attn, V)  # shape: (batch, heads, seq_len, d_k)
        context = context.transpose(1, 2).reshape(batch_size, seq_len, d_model)
        return self.out_proj(context)

In [4]:
# Feed Forward Network
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model)
        )

    def forward(self, x):
        return self.net(x)

In [5]:
# Transformer Encoder Block
class TransformerEncoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff):
        super().__init__()
        self.attn = MultiHeadAttention(d_model, num_heads)
        self.ffn = FeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, x):
        x = self.norm1(x + self.attn(x))
        x = self.norm2(x + self.ffn(x))
        return x

In [16]:
# Full Transformer Encoder
class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, d_ff, num_layers, max_len):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(d_model, max_len)
        self.layers = nn.ModuleList([
            TransformerEncoderBlock(d_model, num_heads, d_ff)
            for _ in range(num_layers)
        ])

    def forward(self, x, attention_mask=None):
        x = self.embedding(x)
        x = self.pos_encoding(x)
        for layer in self.layers:
            x = layer(x)
        return x

In [7]:
vocab_size = 10000
seq_len = 10
batch_size = 2
d_model = 512
num_heads = 8
d_ff = 2048
num_layers = 2
max_len = 100

model = TransformerEncoder(vocab_size, d_model, num_heads, d_ff, num_layers, max_len)
dummy_input = torch.randint(0, vocab_size, (batch_size, seq_len))
output = model(dummy_input)

print(output.shape)  # Expected: (batch_size, seq_len, d_model)

torch.Size([2, 10, 512])


In [18]:
class TransformerClassifier(nn.Module):
    def __init__(self, encoder, hidden_dim, num_classes):
        super().__init__()
        self.encoder = encoder  # your TransformerEncoder
        self.classifier = nn.Linear(hidden_dim, num_classes)

    def forward(self, input_ids, attention_mask=None):
        encoder_output = self.encoder(input_ids)  # pass only input_ids if encoder doesn't support mask
        cls_embedding = encoder_output[:, 0, :]   # assuming [CLS] token is at position 0
        logits = self.classifier(cls_embedding)
        return logits

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [11]:
from transformers import AutoTokenizer

# Define your input sentences
sentences = [
    "The movie was fantastic!",
    "I didn't enjoy the food.",
    "The weather is nice today."
]

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the sentences
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

In [14]:
num_epochs = 5  # or any number of epochs you want to train for

In [20]:
for epoch in range(num_epochs):
    encoder = TransformerEncoder(vocab_size=30522, hidden_dim=768)
    model = TransformerClassifier(encoder, hidden_dim=768, num_classes=2)
    logits = model(input_ids, attention_mask)  # now this works!
    loss = criterion(logits, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

TypeError: TransformerEncoder.__init__() got an unexpected keyword argument 'hidden_dim'

In [ ]:
model.eval()
with torch.no_grad():
    logits = model(input_ids, attention_mask)
    predictions = torch.argmax(logits, dim=1)